# Imports and initialization

In [1]:
import orekit
import math
from org.orekit.propagation import PropagationType
from org.orekit.propagation.events import EventEnablingPredicateFilter, PythonEnablingPredicate, PositionAngleDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder
from org.orekit.propagation.events import EventEnablingPredicateFilter, PythonEnablingPredicate, PositionAngleDetector
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from org.orekit.frames import FramesFactory, LOFType
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.attitudes import LofOffset
from org.hipparchus.geometry.euclidean.threed import Vector3D
from orekit.pyhelpers import setup_orekit_curdir, download_orekit_data_curdir

vm = orekit.initVM()
print('Java version:', vm.java_version)
print('Orekit version:', orekit.VERSION)

download_orekit_data_curdir()
setup_orekit_curdir()

Java version: 1.8.0_152-release
Orekit version: 10.2


# Set up the propagator

In [2]:
date = AbsoluteDate(2020, 10, 11, 0, 0, 0., TimeScalesFactory.getUTC())

orbit = CircularOrbit(800e3 + Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
                      0.01, 0.001, math.radians(40.), math.radians(30), math.radians(80),
                      PositionAngle.MEAN, FramesFactory.getEME2000(), date, Constants.WGS84_EARTH_MU)


builder = DSSTPropagatorBuilder(orbit,
                                DormandPrince853IntegratorBuilder(600.0, 86400.0, 100.0),
                                1.0, PropagationType.MEAN, PropagationType.MEAN)
builder.setMass(1000.)
propagator = builder.buildPropagator(builder.getSelectedNormalizedParameters())
propagator.setEphemerisMode()


# Add the maneuver
This must be where my problem lies, probably when subclassing PythonEnablingPredicate, or when manipulating the parametric types with `of_` ?

In [3]:
class CloseToDatePredicate(PythonEnablingPredicate):
    def __init__(self, date: AbsoluteDate, tolerance_seconds: float):
        self.date = date
        self.tolerance_seconds = tolerance_seconds
        
    def eventIsEnabled(state, eventDetector, g):
        return abs(state.getDate().durationFrom(this.date)) <= this.tolerance_seconds 
    
    
maneuver_approximate_date = AbsoluteDate(2020, 10, 11, 12, 0, 0., TimeScalesFactory.getUTC())
predicate = CloseToDatePredicate(maneuver_approximate_date, 1000.)

position_angle_detector = PositionAngleDetector(OrbitType.CIRCULAR, PositionAngle.TRUE, math.radians(90))
filtered_detector = EventEnablingPredicateFilter(position_angle_detector, predicate)

maneuver_withOUT_filter = ImpulseManeuver(position_angle_detector, LofOffset(FramesFactory.getEME2000(), LOFType.TNW), Vector3D.PLUS_I, 300.)
maneuver_with_filter = ImpulseManeuver(filtered_detector, LofOffset(FramesFactory.getEME2000(), LOFType.TNW), Vector3D.PLUS_I, 300.)

# Propagation

Without maneuver, or with only the `maneuver_withOUT_filter` the propagation works fine.

But when the line adding `maneuver_with_filter` is uncommented, the propagation leads to a NullPointerException on [line 150 of EventEnablingPredicateFilter.java](https://gitlab.orekit.org/orekit/orekit/-/blob/2cf9649809b64986c24245ff47a7bd6a46f29e60/src/main/java/org/orekit/propagation/events/EventEnablingPredicateFilter.java#L150).


In [4]:
final_date = date.shiftedBy(24*3600.)

# position_angle_detector.init(propagator.getInitialState(), final_date) # This is not necessary, proving that issue #663 is fixed
filtered_detector.init(propagator.getInitialState(), final_date) # This does not seem to help, unfortunately

propagator.addEventDetector(maneuver_withOUT_filter)
propagator.addEventDetector(maneuver_with_filter)
propagator.propagate(final_date)

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
java.lang.NullPointerException
	at org.orekit.propagation.events.EventEnablingPredicateFilter.g(EventEnablingPredicateFilter.java:150)
	at org.orekit.forces.maneuvers.ImpulseManeuver.g(ImpulseManeuver.java:153)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator$AdaptedEventDetector.g(AbstractIntegratedPropagator.java:785)
	at org.hipparchus.ode.events.EventState.reinitializeBegin(EventState.java:173)
	at org.hipparchus.ode.AbstractIntegrator.acceptStep(AbstractIntegrator.java:299)
	at org.hipparchus.ode.nonstiff.EmbeddedRungeKuttaIntegrator.integrate(EmbeddedRungeKuttaIntegrator.java:290)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:469)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:414)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.prop